# 1. Gerar Nova Planilha de Mensalidades

Este notebook é responsável pela etapa principal do processamento. Ele realiza a leitura dos dados brutos e atualiza os valores das mensalidades com base na planilha fornecida pela operadora.

**Entradas necessárias:**
- Planilha base (Sheet 'AGO 2025')
- Planilha de mensalidades (Relatório da operadora)

**Saída:**
- Arquivo `dados_atualizados.xlsx`

In [ ]:
import pandas as pd

### 1.1 Carregamento dos Dados
Carrega as planilhas do Excel. 
**Nota:** Certifique-se de preencher os caminhos dos arquivos nas funções `read_excel` abaixo antes de executar.

In [ ]:
                                    # DESCRIÇÃO:                                | EXEMPLO:
caminho_dados = '../Data/'          # Caminho para a planilha de dados          | '../Data/<arquivo>'
caminho_mensalidades = '../Data/'   # Caminho para a planilha de mensalidades   | '../Data/<arquivo>'
mes = ''                            # Mês Atual                                 | 'JAN 2025'

dados = pd.read_excel(caminho_dados, skiprows=1, sheet_name=mes)
mensalidades = pd.read_excel(caminho_mensalidades, index_col=0)

### 1.2 Visualização Inicial
Exibe as primeiras linhas para garantir que os dados foram carregados corretamente.

In [ ]:
display(dados.head())
display(mensalidades.head())

### 1.3 Atualização dos Valores
Itera sobre a planilha de mensalidades e atualiza a coluna `MENSALIDADE` na planilha principal (`dados`) correspondente ao nome encontrado.

**Melhoria:** Realiza limpeza de espaços nos nomes para garantir melhor correspondência.

In [ ]:
def normaliza_codigo(valor):
    if pd.isna(valor):
        return pd.NA
    texto = str(valor).strip()
    if texto == '' or texto.lower() == 'nan':
        return pd.NA
    digits = ''.join(ch for ch in texto if ch.isdigit())
    return digits if digits else pd.NA


dados['NOME'] = dados['NOME'].astype('string').str.replace(r"\s+", " ", regex=True).str.strip()
mensalidades['Nome'] = mensalidades['Nome'].astype('string').str.replace(r"\s+", " ", regex=True).str.strip()

mensalidades_validas = mensalidades[mensalidades['Nome'].notna() & (mensalidades['Nome'] != '')].copy()
if 'Tipo Valor' in mensalidades_validas.columns:
    mensalidades_validas['Tipo Valor'] = mensalidades_validas['Tipo Valor'].astype('string').str.strip()
    mensalidades_validas = mensalidades_validas[mensalidades_validas['Tipo Valor'] == 'Mensalidade']

mensalidades_validas['Codigo'] = mensalidades_validas.index.map(normaliza_codigo)

dados_codigos = dados['CARTEIRA'].apply(normaliza_codigo)
dados_nomes = dados['NOME']

contador_atualizados = 0
nomes_multiplos = []
nomes_nao_encontrados = []

for index, row in mensalidades_validas.iterrows():
    codigo = row['Codigo']
    nome_mensalidade = row['Nome']
    valor_mensalidade = row['Valor']

    codigo_valido = pd.notna(codigo) and str(codigo).strip() != ''

    if codigo_valido:
        mask_codigo = (dados_codigos == codigo).fillna(False)
    else:
        mask_codigo = None

    if codigo_valido and mask_codigo.any() and mask_codigo.sum() == 1:
        dados.loc[mask_codigo, 'MENSALIDADE'] = valor_mensalidade
        contador_atualizados += 1
        continue

    if nome_mensalidade and nome_mensalidade in dados_nomes.values:
        matches = dados_nomes == nome_mensalidade
        if matches.sum() == 1:
            dados.loc[matches, 'MENSALIDADE'] = valor_mensalidade
            contador_atualizados += 1
        else:
            nomes_multiplos.append(nome_mensalidade)
    elif nome_mensalidade:
        nomes_nao_encontrados.append(nome_mensalidade)

print(f"Total de registros atualizados/preenchidos: {contador_atualizados}")
if nomes_multiplos:
    print("ATENÇÃO: nomes repetidos em dados (não atualizados por nome):")
    for nome in sorted(set(nomes_multiplos)):
        print(f"- {nome}")
else:
    print("Sucesso: nenhum nome repetido detectado na atualização.")

if nomes_nao_encontrados:
    print("ATENÇÃO: nomes não encontrados em dados:")
    for nome in sorted(set(nomes_nao_encontrados)):
        print(f"- {nome}")
else:
    print("Sucesso: nenhum nome ausente na atualização.")

display(dados.head())

### 1.4 Exportação
Salva o resultado processado em um novo arquivo Excel.

In [ ]:
dados.to_excel('../Data/dados_atualizados.xlsx', index=False)